In [7]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import json     # <-- NEW
import base64   # <-- NEW

# --- 1. Setup Your Inputs ---
BLANK_IMAGE_PATH = '../preprocessor/question_paper_templates/1.jpg'
FILLED_IMAGE_FOLDER = '../preprocessor/aligned_outputs'

# --- 2. Create Output Directories ---
os.makedirs("evaluation_results", exist_ok=True) # For debug images
os.makedirs("agent1_output", exist_ok=True)      # <-- NEW: For Agent 2's data

# --- 3. Automatically Find All Image Paths ---
print(f"Scanning for images in: {FILLED_IMAGE_FOLDER}")
image_extensions = ('*.jpg', '*.jpeg', '*.png')
FILLED_IMAGE_PATHS = []
for ext in image_extensions:
    FILLED_IMAGE_PATHS.extend(glob.glob(os.path.join(FILLED_IMAGE_FOLDER, ext)))

if not FILLED_IMAGE_PATHS:
    print(f"FATAL ERROR: No images found in {FILLED_IMAGE_FOLDER}")
    exit()

print(f"Found {len(FILLED_IMAGE_PATHS)} images to process.")


# --- 4. Load and Pre-process BLANK Image (Once) ---
print(f"\nLoading blank reference image: {BLANK_IMAGE_PATH}")
img_blank = cv2.imread(BLANK_IMAGE_PATH)

if img_blank is None:
    print(f"FATAL ERROR: Could not read blank image at {BLANK_IMAGE_PATH}")
    exit()

gray_blank = cv2.cvtColor(img_blank, cv2.COLOR_BGR2GRAY)
h, w = gray_blank.shape
gray_blank = cv2.GaussianBlur(gray_blank, (5,5), 0)
print("Blank image processed successfully.")


# --- 5. Start Loop to Process Each Answer Sheet ---
print("\n--- Starting batch processing ---")
for image_path in FILLED_IMAGE_PATHS:
    print(f"\nProcessing image: {image_path}")
    
    # --- 5a. Load FILLED Image ---
    img_filled = cv2.imread(image_path)
    if img_filled is None:
        print(f"Skipping image, could not be loaded.")
        continue 

    # --- 5b. Pre-process FILLED Image ---
    # Create the resized color image for Agent 2
    img_filled_resized = cv2.resize(img_filled.copy(), (w, h))
    
    # Create the grayscale version for diff processing
    gray_filled = cv2.cvtColor(img_filled_resized, cv2.COLOR_BGR2GRAY)
    gray_filled = cv2.GaussianBlur(gray_filled, (5,5), 0)
    
    # Create the image for drawing boxes
    img_with_boxes = img_filled_resized.copy()

    # --- 5c. Find Differences ---
    diff = cv2.absdiff(gray_blank, gray_filled)

    # --- 5d. Threshold and Clean Up ---
    _, thresh = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)
    kernel = np.ones((7,7), np.uint8)
    clean = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)
    kernel_h = np.ones((5, 100), np.uint8)
    merged_regions = cv2.dilate(clean, kernel_h, iterations=1)

    # --- 5e. Find and Sort Regions ---
    contours, _ = cv2.findContours(merged_regions, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    bounding_boxes = []
    for c in contours:
        x, y, w_c, h_c = cv2.boundingRect(c)
        if (w_c * h_c) > 100:
            # Convert numpy int32 to standard int for JSON
            bounding_boxes.append((int(x), int(y), int(w_c), int(h_c))) 

    bounding_boxes.sort(key=lambda box: box[1])

    # --- 5f. Output Coordinates and Visualize ---
    print(f"Found {len(bounding_boxes)} answer regions:")
    for j, (x, y, w_box, h_box) in enumerate(bounding_boxes):
        print(f"  Region {j+1}: [x={x}, y={y}, w={w_box}, h={h_box}]")
        cv2.rectangle(img_with_boxes, (x, y), (x+w_box, y+h_box), (0, 255, 0), 2)
        cv2.putText(img_with_boxes, str(j+1), (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # --- 5g. Save Debug Image ---
    base_name = os.path.basename(image_path)
    file_name_only = os.path.splitext(base_name)[0]
    output_filename = f"evaluation_results/{file_name_only}_result.png"
    
    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(img_with_boxes, cv2.COLOR_BGR2RGB))
    plt.title(f"Detected Regions for {base_name}")
    plt.axis("off")
    plt.savefig(output_filename)
    plt.close()
    print(f"Saved debug image to {output_filename}")

    # --- 5h. NEW: Save Data for Agent 2 ---
    # 1. Encode the resized color image to base64
    _, buffer = cv2.imencode('.jpg', img_filled_resized)
    image_base64 = base64.b64encode(buffer).decode('utf-8')
    
    # 2. Create the data packet
    data_for_agent_2 = {
        "image_base64": image_base64,
        "rois": bounding_boxes
    }
    
    # 3. Save as a JSON file
    json_filename = f"agent1_output/{file_name_only}_data.json"
    with open(json_filename, 'w') as f:
        json.dump(data_for_agent_2, f)
        
    print(f"Saved data for Agent 2 to {json_filename}")

print("\n--- Batch processing complete. ---")

Scanning for images in: ../preprocessor/aligned_outputs
Found 2 images to process.

Loading blank reference image: ../preprocessor/question_paper_templates/1.jpg
Blank image processed successfully.

--- Starting batch processing ---

Processing image: ../preprocessor/aligned_outputs/1 copy.jpg
Found 7 answer regions:
  Region 1: [x=1384, y=920, w=184, h=56]
  Region 2: [x=1408, y=1108, w=138, h=82]
  Region 3: [x=1403, y=1278, w=146, h=59]
  Region 4: [x=1393, y=1420, w=151, h=49]
  Region 5: [x=1392, y=1577, w=156, h=52]
  Region 6: [x=1405, y=1935, w=115, h=28]
  Region 7: [x=986, y=2091, w=165, h=57]
Saved debug image to evaluation_results/1 copy_result.png
Saved data for Agent 2 to agent1_output/1 copy_data.json

Processing image: ../preprocessor/aligned_outputs/1-2 copy.jpg
Found 7 answer regions:
  Region 1: [x=1401, y=929, w=150, h=54]
  Region 2: [x=1405, y=1121, w=138, h=53]
  Region 3: [x=1414, y=1271, w=131, h=65]
  Region 4: [x=1399, y=1416, w=135, h=55]
  Region 5: [x=1396